# Dev: Model to Form

## Preamble

In [1]:
import ipywidgets as widgets

In [2]:
from pathlib import Path
from typing import Type

In [3]:
import sqlmodel

In [4]:
import tuttle

2022-01-10 10:14:49,488 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-10 10:14:49,489 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-01-10 10:14:49,489 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,489 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-01-10 10:14:49,490 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,490 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bank")
2022-01-10 10:14:49,490 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,490 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bankaccount")
2022-01-10 10:14:49,491 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,491 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("contact")
2022-01-10 10:14:49,491 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,491 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("client")
2022-01-10 10:14:49,492 INFO sqlalchemy.engine.Engine [raw sql] ()
20

In [5]:
from tuttle import model

In [6]:
import collections

In [7]:
import beepy

## Setup

In [8]:
test_home = Path("tmp")
db_path = test_home / "tuttle_test.db"
db_url = f"sqlite:///{db_path}"
db_engine = sqlmodel.create_engine(db_url, echo=True)
sqlmodel.SQLModel.metadata.create_all(db_engine)

2022-01-10 10:14:49,617 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-01-10 10:14:49,618 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-01-10 10:14:49,621 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,622 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-01-10 10:14:49,623 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,624 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bank")
2022-01-10 10:14:49,624 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,625 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bankaccount")
2022-01-10 10:14:49,626 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,627 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("contact")
2022-01-10 10:14:49,627 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-01-10 10:14:49,627 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("client")
2022-01-10 10:14:49,628 INFO sqlalchemy.engine.Engine [raw sql] ()
20

## Workflows

### Get User Name and E-Mail

In [9]:
model.User?

Init signature:
model.User(
    *,
    id: int = None,
    name: str,
    subtitle: str,
    business_name: str = None,
    email: pydantic.networks.EmailStr,
    VAT_number: str,
) -> None
File:           ~/Documents/Work/Projects/PrototypeFund/Dev/tuttle/tuttle/model.py
Type:           SQLModelMetaclass
Subclasses:     


In [10]:
model.User(
    name="John Doe",
    email="john@doe.com"
)

User(id=None, name='John Doe', business_name=None, email='john@doe.com')

In [11]:
model.User.schema()["properties"]

{'id': {'title': 'Id', 'type': 'integer'},
 'name': {'title': 'Name', 'type': 'string'},
 'subtitle': {'title': 'Subtitle', 'type': 'string'},
 'business_name': {'title': 'Business Name', 'type': 'string'},
 'email': {'title': 'Email', 'type': 'string', 'format': 'email'},
 'VAT_number': {'title': 'Vat Number', 'type': 'string'}}

In [12]:
def model_form(
    model_class: Type[sqlmodel.SQLModel]
):
    properties = model_class.schema()["properties"]
    form_header = [
        widgets.HTML(
            value=f"<b>Create {model_class.__name__}</b>",
        )
    ]
    form_elements = collections.OrderedDict()
    for (prop_name, prop_meta) in properties.items():
        if prop_name == "id":
            # ignore id
            pass
        if prop_meta["type"] == "string":
            text_field = widgets.Text(
                placeholder='...',
                description=prop_meta["title"],
                disabled=False
            )
            form_elements[prop_name] = text_field
    # assemble form

    create_button = widgets.Button(
        description='Create',
        disabled=False,
        button_style='success', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Create model object',
        icon="hammer" # (FontAwesome names without the `fa-` prefix)
    )
    
    def factory_function():
        instance = model_class(
            **dict(
                (prop_name, form_elements[prop_name].value) 
                for prop_name in form_elements.keys()
            )
        )
        beepy.beep()
        return instance
   

    create_button.on_click(factory_function)
    
    form = widgets.VBox(
        children=(
            form_header
            + list(form_elements.values())
            + [create_button]
        )
    )
    return form

In [13]:
model_form(model.BankAccount)